<a href="https://colab.research.google.com/github/Ezra-11/knowledge-level-of-wikipedia/blob/main/DataMining_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Business Understanding

## 1.1 Problem Statement
Wikipedia content about *Zambia* remains significantly underrepresented, with very few comprehensive pages dedicated to Zambian topics, history, culture, and current affairs. The *DataLab Research group* at The University of Zambia has identified that understanding the knowledge levels and expertise of current Wikipedia contributors working on Zambian content is crucial for developing targeted strategies to improve content quality and quantity.

Currently, there is no systematic way to assess the expertise and knowledge depth of contributors to Zambian Wikipedia pages, making it difficult to:

- Identify subject matter experts who could mentor new contributors  
- Match contributors with appropriate content areas based on their expertise  
- Develop targeted training programs for contributors with different knowledge levels  
- Optimize content review and quality assurance processes  

*Core Problem:* How can we systematically classify and understand the knowledge levels of Wikipedia contributors working on Zambian content to improve the overall quality and quantity of Wikipedia pages about Zambia?

## 1.2 Business Objectives

### Primary Objectives:
- *Enhance Content Quality:* Improve the accuracy, depth, and comprehensiveness of Wikipedia articles about Zambia by better understanding contributor expertise levels  
- *Optimize Contributor Engagement:* Develop targeted strategies to engage contributors based on their knowledge levels and areas of expertise  
- *Facilitate Knowledge Transfer:* Identify expert contributors who can mentor newcomers and provide guidance on complex topics  
- *Improve Content Coverage:* Strategically assign content creation and improvement tasks based on contributor knowledge levels  



 Secondary Objectives:
- **Build Community Capacity:** Create a framework for assessing and developing Wikipedia contributor skills within the Zambian context  
- ***Inform Training Programs:*** Develop evidence-based training curricula tailored to different contributor knowledge levels  
- ***Support Academic Research:*** Provide insights into digital knowledge creation patterns in developing countries  

### Success Metrics:
- *Short-term:* Accurate classification of contributor knowledge levels with measurable confidence scores  
- *Medium-term:* Increased quality scores of Zambian Wikipedia articles (measured by completeness, references, and peer ratings)  
- ***Long-term:*** Growth in the number of active Zambian Wikipedia contributors and articles about Zambia  





##**1.3 Data Mining Goals**

Primary Data Mining Goal:
Build a *multi-class classification model* that can automatically categorize Wikipedia contributors working on Zambian content into distinct knowledge level categories (e.g., Novice, Intermediate, Advanced, Expert) based on their contribution patterns, edit quality, and content expertise.

###Specific Technical Objectives

Feature Engineering
Extract meaningful features from contributor data including:
- Edit frequency and consistency patterns  
- Quality indicators (citations added, grammar improvements, factual accuracy)  
- Content complexity (technical terms, specialized knowledge areas)  
- Collaboration patterns (talk page participation, peer interactions)  
- Longevity and commitment indicators  

###Classification Model Development
- Implement and compare multiple classification algorithms (Random Forest, SVM, Neural Networks, etc.)  
- Develop ensemble methods to improve classification accuracy  
- Create interpretable models that explain why a contributor is classified at a particular level  

###Clustering Analysis (Secondary)
- Identify natural groupings of contributors based on their expertise areas and contribution styles  
- Discover contributor archetypes that may not align with traditional knowledge level categories  

###Predictive Modeling
- Predict potential contributor knowledge growth trajectories  
- Identify contributors likely to become long-term, high-quality contributors

##**1.4 Project Success Criteria**

###Model Performance Criteria
- *Primary Metric:* Classification accuracy of at least *80%* on a held-out test set  
- *Precision and Recall:* Minimum *75%* precision and recall for each knowledge level category  
- *F1-Score:* Overall weighted F1-score of at least *0.78*  
- *Cross-validation Stability:* Model performance should not vary by more than *5%* across different data splits  

###Business Impact Criteria
- *Expert Validation:* At least *85%* agreement between model classifications and manual assessments by domain experts  
- *Actionability:* Classification results must provide clear, actionable insights for contributor development strategies  
- *Scalability:* Model should be able to process new contributors and updates efficiently (*< 1 minute* per contributor)  

###Technical Criteria
- *Interpretability:* Model decisions must be explainable with feature importance scores and decision reasoning  
- *Robustness:* Model should maintain performance when applied to contributors from different time periods  
- *Generalizability:* Framework should be adaptable to other developing country contexts or subject areas  

###Data Quality Criteria
- *Coverage:* Successfully classify at least *90%* of active contributors to Zambian Wikipedia pages  
- *Consistency:* Inter-rater reliability (Kappa score) *> 0.7* for manual validation labels  
- *Completeness:* Less than *10%* missing data across critical features  

###Deployment Criteria
- *Integration:* Model should be implementable within existing Wikipedia contributor management workflows  
- *User Acceptance:* Positive feedback from at least *80%* of Wikipedia administrators and experienced contributors who test the system  
- *Maintenance:* Established process for model updates and performance monitoring over time